### Install package

In [1]:
!pip install pytorch-adapt[lightning]

     |████████████████████████████████| 137 kB 4.4 MB/s 
     |████████████████████████████████| 107 kB 37.2 MB/s 
     |████████████████████████████████| 397 kB 35.9 MB/s 
     |████████████████████████████████| 527 kB 41.0 MB/s 
     |████████████████████████████████| 952 kB 35.9 MB/s 
     |████████████████████████████████| 829 kB 38.0 MB/s 
     |████████████████████████████████| 134 kB 51.7 MB/s 
     |████████████████████████████████| 596 kB 45.3 MB/s 
     |████████████████████████████████| 1.1 MB 32.0 MB/s 
     |████████████████████████████████| 271 kB 51.6 MB/s 
     |████████████████████████████████| 94 kB 639 kB/s 
     |████████████████████████████████| 144 kB 51.8 MB/s 
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sha256=48ca28fc5d96e39a7b3c7c4415680df7d70b233b0c7ba4cadcf2bc3b50e479a8
  Stored in directory: /root/.cache/pip/wheels/56/b0/fe/4410d17b32f1f0c3cf54cdfb2bc04d7b4b8f4ae377e2229ba0
Successfully built future
  Attempting uninstall

### Import packages

In [2]:
import pytorch_lightning as pl
import torch

from pytorch_adapt.adapters import DANN
from pytorch_adapt.containers import Models, Optimizers
from pytorch_adapt.datasets import DataloaderCreator, get_mnist_mnistm
from pytorch_adapt.frameworks.lightning import Lightning
from pytorch_adapt.frameworks.utils import filter_datasets
from pytorch_adapt.models import Discriminator, mnistC, mnistG
from pytorch_adapt.validators import IMValidator

### Create datasets and dataloaders

In [3]:
datasets = get_mnist_mnistm(["mnist"], ["mnistm"], folder=".", download=True)
dc = DataloaderCreator(batch_size=32, num_workers=2)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw



  0%|          | 0/134178716 [00:00<?, ?it/s]

100%|██████████| 68007/68007 [00:16<00:00, 4169.63it/s]


### Create models, optimizers, hook, and validator

In [4]:
G = mnistG(pretrained=True)
C = mnistC(pretrained=True)
D = Discriminator(in_size=1200, h=256)
models = Models({"G": G, "C": C, "D": D})
optimizers = Optimizers((torch.optim.Adam, {"lr": 0.0001}))

adapter = DANN(models=models, optimizers=optimizers)
validator = IMValidator()
dataloaders = dc(**filter_datasets(datasets, validator))
train_loader = dataloaders.pop("train")

L_adapter = Lightning(adapter, validator=validator)
trainer = pl.Trainer(gpus=1, max_epochs=2)

Downloading: "https://cornell.box.com/shared/static/tdx0ts24e273j7mf3r2ox7a12xh4fdfy" to /root/.cache/torch/hub/checkpoints/mnistG-68ee7945.pth


  0%|          | 0.00/161k [00:00<?, ?B/s]

Downloading: "https://cornell.box.com/shared/static/j4zrogronmievq1csulrkai7zjm27gcq" to /root/.cache/torch/hub/checkpoints/mnistC-ac7b5a13.pth


  0%|          | 0.00/1.31M [00:00<?, ?B/s]

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


### Train and evaluate

In [5]:
trainer.fit(L_adapter, train_loader, list(dataloaders.values()))

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Missing logger folder: /content/lightning_logs

  | Name   | Type       | Params
--------------------------------------
0 | models | ModuleDict | 756 K 
1 | misc   | ModuleDict | 0     
--------------------------------------
756 K     Trainable params
0         Non-trainable params
756 K     Total params
3.024     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]